## 要讓 Python 寄信，需先申請**應用程式密碼**

## 要申請應用程式密碼，必須先開通二步驟認證。

### 登入 Google 帳戶 --> Goolge 帳戶


![GoogleLogin.PNG](images/GoogleLogin.PNG)

![GoogleLogin2.PNG](images/GoogleLogin2.PNG)

![GoogleLogin5.PNG](images/GoogleLogin5.PNG)


### 自訂應用程式

![GoogleLogin3.PNG](images/GoogleLogin3.PNG)

![GoogleLogin4.PNG](images/GoogleLogin4.PNG)


# 跟郵件伺服器打招呼

In [5]:
import smtplib  
smtpssl=smtplib.SMTP_SSL("smtp.gmail.com", 465) 
smtp.ehlo()

(451, b'4.4.2 Timeout - closing connection. q12sm68473960pfh.158 - gsmtp')

# 嘗試用帳號密碼登入 -- 用密碼登入會失敗

In [4]:
import smtplib 
smtpssl=smtplib.SMTP_SSL("smtp.gmail.com", 465) 
smtpssl.login("mailtest1458@gmail.com", "xxxxxxxx")

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials z30sm71614044pfq.154 - gsmtp')

# oops! 出錯了!

### 必須進入 Google 帳號管理為應用程式產生特別的密碼才行

但要注意! 帳戶要先設定兩階段驗證後才會出現 應用程式密碼。

在彈出頁面中顯示了一組應用程式專用密碼, 須立即複製到記事本中儲存, 因為只要關閉此頁面便無法查出此密碼, 只能刪除再重新產生 :

    wbpwiranswgebhvq


# 嘗試用「應用程式密碼」登入伺服器
![GoogleLogin4.PNG](images/GoogleLogin4.PNG)


In [1]:
import smtplib 
smtpssl=smtplib.SMTP_SSL("smtp.gmail.com", 465) 
smtpssl.login("mailtest1458@gmail.com", "wbpwiranswgebhvq")

(235, b'2.7.0 Accepted')

# 寄一封信試試看

In [2]:
import smtplib 
smtpssl=smtplib.SMTP_SSL("smtp.gmail.com", 465) 
smtpssl.login("mailtest1458@gmail.com", "wbpwiranswgebhvq")
from_addr = 'mailtest1458@gmail.com'
to_addr = 'jiangsir@tea.nknush.kh.edu.tw'
msg = 'Subject:Python Test mail!!!\n'
msg += 'test mail content'
smtpssl.sendmail(from_addr, to_addr, msg, mail_options=(), rcpt_options=())
smtpssl.quit() 

(221, b'2.0.0 closing connection b14sm9903339pfi.92 - gsmtp')

# 寄送中文信件 -- 用 MIME 協定

解決此問題的辦法是使用多用途郵件擴展協定 MIME (Multiplepurpose Internet Mail Extensions), 

In [21]:
import smtplib 
from email.mime.text import MIMEText  

smtpssl=smtplib.SMTP_SSL("smtp.gmail.com", 465) 
smtpssl.login("mailtest1458@gmail.com", "wbpwiranswgebhvq")
msg = 'test mail 中文內容測試'
mime=MIMEText(msg, "plain", "utf-8")  
mime["Subject"]="Python中文信件!!!(MIME)"

# 顯示的名稱
mime["From"]="我是誰?"
mime["To"]="jiangsir@tea.nknush.kh.edu.tw"

# 寄件者，收件者
from_addr = 'mailtest1458@gmail.com'
to_addr = 'jiangsir@tea.nknush.kh.edu.tw'

smtpssl.sendmail(from_addr, to_addr, mime.as_string())
smtpssl.quit() 

(221, b'2.0.0 closing connection s21sm38823568pfm.3 - gsmtp')

# 傳送 HTML 信件 -- 使用 MIME 協定

In [3]:
import smtplib
from email.mime.text import MIMEText

html="""
<!doctype html>
<html>
<head>
  <meta charset='utf-8'>
  <title>HTML mail</title>
</head>
<body>
  <b>HTML 郵件測試</b>
</body>
</html>
"""
mime=MIMEText(html, "html", "utf-8")
mime["Subject"]="寄送HTML 郵件(MIME)"
mime["From"]="我是誰?"
mime["To"]="你是誰?"
msg=mime.as_string()

smtpssl=smtplib.SMTP_SSL("smtp.gmail.com", 465) 
smtpssl.login("mailtest1458@gmail.com", "wbpwiranswgebhvq")

from_addr="mailtest1458@gmail.com"
to_addr=["jiangzero@gmail.com"]
status=smtpssl.sendmail(from_addr, to_addr, msg)
if status=={}:
    print("郵件傳送成功!")
else:
    print("郵件傳送失敗!")
smtpssl.quit()

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials a31sm44474857pgb.93 - gsmtp')

## 郵寄附件 -- 以 Bebras 2020 挑戰證明群發為例

In [2]:
import os
import sys
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


COMMASPACE = ', '
subject = '高師大附中 109學年度國際運算思維挑戰賽 挑戰證明'
from_addr = "mailer@zerojudge.tw"
app_passwd = "zridfkvaolgtoekp"
to_addr = [""]
to_domain = 'gmail.com' # email domain: stu.nknush.kh.edu.tw

# attachePath = 'C:\\Users\\user\\Downloads\\109Bebras成績證明\\'
attachePath = '/mnt/c/Users/user/Downloads/109Bebras成績證明/'
attachments = sorted(os.listdir(attachePath))  # 取得成績證明資料夾的所有檔案
#attachments = [attachePath+name for name in os.listdir(attachePath)]


def main():
    # html 格式的信件內容
    htmlcontent = """
    <!doctype html>
    <html>
    <head>
    <meta charset='utf-8'>
    <title>HTML mail</title>
    </head>
    <body>
    <b>運算思維挑戰賽挑戰證明</b>
    </body>
    </html>
    """

    # 讀取所有的挑戰證明檔案
    for filename in attachments:
        # 建立郵件主題
        content = MIMEMultipart()  # 建立MIMEMultipart物件
        content["subject"] = subject  # 郵件標題
        content["from"] = from_addr  # 寄件者
        to_addr[0] = filename.split('.')[0] + to_domain  # 收件者 email
        content["to"] = to_addr[0]  # 收件者

        content.attach(MIMEText(htmlcontent, 'html'))

        try:
            with open(attachePath+filename, 'rb') as fp:
                print('讀取 filename=', fp.name)
                msg = MIMEBase('application', "octet-stream")
                msg.set_payload(fp.read())

            encoders.encode_base64(msg)
            msg.add_header('Content-Disposition', 'attachment',
                           filename=os.path.basename(filename))
            content.attach(msg)

        except:
            print("Unable to open one of the attachments. Error: ",
                  sys.exc_info()[0])
            raise

        # 寄送EMAIL
        smtpssl = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        smtpssl.login(from_addr, app_passwd)
        print('sendmail from', from_addr, 'to', to_addr, end="")
        status = smtpssl.sendmail(from_addr, to_addr, content.as_string())
        smtpssl.quit()
        if status == {}:
            print(' --- 郵件傳送成功!')
        else:
            print(' --- 失敗', status)


if __name__ == '__main__':
    main()


can read faile filename= 910181.pdf fp= /mnt/c/Users/user/Downloads/109Bebras成績證明/910181.pdf
sendmail from mailer@zerojudge.tw to ['910181@stu.nknush.kh.edu.tw'] --- 郵件傳送成功!
can read faile filename= 910182.pdf fp= /mnt/c/Users/user/Downloads/109Bebras成績證明/910182.pdf
sendmail from mailer@zerojudge.tw to ['910182@stu.nknush.kh.edu.tw'] --- 郵件傳送成功!
can read faile filename= 910183.pdf fp= /mnt/c/Users/user/Downloads/109Bebras成績證明/910183.pdf
sendmail from mailer@zerojudge.tw to ['910183@stu.nknush.kh.edu.tw'] --- 郵件傳送成功!
can read faile filename= 910184.pdf fp= /mnt/c/Users/user/Downloads/109Bebras成績證明/910184.pdf
sendmail from mailer@zerojudge.tw to ['910184@stu.nknush.kh.edu.tw'] --- 郵件傳送成功!
can read faile filename= 910185.pdf fp= /mnt/c/Users/user/Downloads/109Bebras成績證明/910185.pdf
sendmail from mailer@zerojudge.tw to ['910185@stu.nknush.kh.edu.tw'] --- 郵件傳送成功!
can read faile filename= 910186.pdf fp= /mnt/c/Users/user/Downloads/109Bebras成績證明/910186.pdf
sendmail from mailer@zerojudge.tw to 

### 參考文獻

http://yhhuang1966.blogspot.com/2018/10/python-gmail.html
